In [ ]:
import json
from tqdm import tqdm 
import random 

In [ ]:
random.seed(0)
# 直接从equal_split_word数据生成,考虑了coarse的情况
attr_dict_file = '../../data/equal_processed_data/attr_to_attrvals.json'
with open(attr_dict_file, 'r') as f:
    attr_dict = json.load(f)

def get_negative_dict(attr_dict):
        negative_dict = {}
        for query, attr_list in attr_dict.items():
            negative_dict[query] = {}
            for attr in attr_list:
                l = attr_list.copy()
                l.remove(attr)
                negative_dict[query][attr] = l
        return negative_dict
negative_dict = get_negative_dict(attr_dict)


# file = '../../data/equal_split_word/attr/coarse4588.txt'
# save_file = '../../data/equal_split_word/mytest4588_hide.txt'
file = '../../data/equal_split_word/fine5000.txt'
save_file = '../../data/equal_split_word/mytest5000_hide.txt'


rets = []
with open(file, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        img_name = data['img_name']
        title = data['title']
        key_attr = data['key_attr']
        if not key_attr:
            continue
        match = data['match']
        vocab_split = data['vocab_split']
        del data['title_split']
        # del data['feature']
        # 包 永远只有一个属性，1/3的概率为负
        if '类别' in key_attr.keys():
            if random.random() < 1/3:
                query = '类别'
                attr = key_attr[query]
                new_attr = random.sample(negative_dict[query][attr], 1)[0]
                key_attr[query] = new_attr
                title = title.replace(attr, new_attr)
                idx = vocab_split.index(attr)
                vocab_split[idx] = new_attr
                match['类别'] = 0
                match['图文'] = 0
        # 其他样本，29.2%的概率为正样本
        else:
            if random.random() < (1 - 0.292): # 以此概率为负
                match['图文'] = 0
                # 先随机选取一个属性为负
                query = random.sample(list(key_attr.keys()), 1)[0]
                attr = key_attr[query]
                new_attr = random.sample(negative_dict[query][attr], 1)[0]
                key_attr[query] = new_attr
                title = title.replace(attr, new_attr)
                idx = vocab_split.index(attr)
                vocab_split[idx] = new_attr
                match[query] = 0
                # 其余属性以一定概率变负
                for query, attr in key_attr.items():
                    if match[query] != 0:
                        if random.random() < 0.434: # 0.434
                            new_attr = random.sample(negative_dict[query][attr], 1)[0]
                            key_attr[query] = new_attr
                            title = title.replace(attr, new_attr)
                            idx = vocab_split.index(attr)
                            vocab_split[idx] = new_attr
                            match[query] = 0
                # 隐藏属性
                if len(key_attr) >= 2: # 属性不少于两个
                    flag_0 = 0
                    flag_1 = 0
                    for query, value in match.items(): # 有正有负
                        if query != '图文':
                            if value == 1:
                                flag_1 = 1
                            else:
                                flag_0 = 1
                    if flag_0 and flag_1:
                        if random.random() < 0.3: # 一定概率进行隐藏
                            match_copy = match.copy()
                            for query, value in match_copy.items():
                                if query != '图文':
                                    if value == 0:
                                        del match[query]
                                        del key_attr[query]
            
        data['title'] = title 
        data['key_attr'] = key_attr 
        data['match'] = match 
        data['vocab_split'] = vocab_split
        rets.append(json.dumps(data, ensure_ascii=False)+'\n')

print(len(rets))
with open(save_file, 'w') as f:
    f.writelines(rets)

In [ ]:
# 生成只包含match的ground_truth文件

# file = '../../data/equal_split_word/mytest4588_hide.txt'
# save_file = '../../data/equal_split_word/mytest4588_hide_gt.txt'

file = '../../data/equal_split_word/mytest5000_hide.txt'
save_file = '../../data/equal_split_word/mytest5000_hide_gt.txt'

rets = []
with open(file, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        del data['title']
        del data['key_attr']
        del data['feature']
        del data['vocab_split']
        rets.append(json.dumps(data, ensure_ascii=False)+'\n')

print(len(rets))
with open(save_file, 'w') as f:
    f.writelines(rets)

In [3]:
import json

with open('./data/fl_split_word/vocab/vocab_dict.json', 'r') as f:
    word_dict = json.load(f)
print('亨利领' in word_dict)

True
